In [1]:
import findspark
findspark.init()

In [2]:
import yaml
import os

import pandas as pd
import numpy as np
import tensorflow as tf

from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.types import StructType
from pyspark.sql.functions import col, to_date, to_timestamp
from pyspark.errors import PySparkException
from pymongo import MongoClient
from pymongo import errors

2024-03-06 19:53:25.615455: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [3]:
def gen_mongo_uri(mongo_conf) -> str | None:
    """
    generate mongo connection uri based on input
    :return:
    """
    if not mongo_conf:
        return None
    return (f"mongodb+srv://{mongo_conf['user']}:"
            f"{mongo_conf['token']}@"
            f"{mongo_conf['host']}"
            f"/?retryWrites=true&w=majority")

def get_property(conf) -> dict:
    """
    get property for database
    """
    return {key: conf[key] for key in conf.keys()
                         & {'user', 'password', 'driver'}}

def gen_maria_jdbc(conf, db: str = '') -> str | None:
    """
    get connection jdbc string for maria database
    """
    if not conf:
        return None

    db = conf['database'] if not db else db
    return (f"jdbc:mysql://{conf['host']}:"
            f"{conf['port']}/"
            f"{db}?permitMysqlScheme")

def gen_postgres_jdbc(conf, db: str = '') -> str | None:
    """
    get connection jdbc string for maria database
    """
    if not conf:
        return None
    db = conf['database'] if not db else db
    return (f"jdbc:postgresql://{conf['host']}:"
            f"{conf['port']}/{db}")

def gen_mssql_jdbc(conf, db: str = '') -> str | None:
    """
    get connection jdbc string for maria database
    """
    if not conf:
        return None
    db = conf['database'] if not db else db
    return (f"jdbc:sqlserver://{conf['host']}:{conf['port']};"
            f"databaseName={db};encrypt=true;trustServerCertificate=true;")

def gen_azure_jdbc(conf, db: str = '') -> str | None:
    """
    get connection jdbc string for azure sql database
    """
    if not conf:
        return None
    db = conf['database'] if not db else db
    return (f"jdbc:sqlserver://{conf['host']}:{conf['port']};"
            f"databaseName={db};encrypt=true;")


def init_mongodb_client(uri: str) -> MongoClient | None:
    """
    initialize the mongo client
    """
    try:
        # Initialize new MongoDB client
        client = MongoClient(uri)
    except errors.ConnectionFailure as e:
        # Handle connection failure gracefully
        print(f"Failed to connect to MongoDB: {e}")

        return None
    else:
        return client

def prepare_dataframe(spark, sc, data) -> DataFrame:
    """
    prepare dataframe when the data might be in different data types
    :param spark: spark session
    :param sc: spark context
    :param data: data to be processed
    :return: spark sql dataframe
    """
    if isinstance(data, pd.DataFrame) and not data.empty:
        print('data is pandas df and not empty')
        df = spark.createDataFrame(data)
    elif isinstance(data, DataFrame) and not data.isEmpty():
        print('data is spark df and not empty')
        df = data
    elif isinstance(data, list):
        print('data is list and not empty')
        df = spark.read.json(sc.parallelize([json.dumps(record) for record in data]))
    elif isinstance(data, dict):
        print('data is dictionary and not empty')
        data = [data]
        df = spark.read.json(sc.parallelize([json.dumps(record) for record in data]))
    else:
        
        print('data empty')
        # initialize empty dataframe
        schema = StructType([])
        df = spark.createDataFrame([], schema)

    return df
    
def write_to_mongo(spark, data, 
                   uri: str, db: str, col: str) -> None:
    """
    write data into mongo db
    :param spark: sparkSession
    :param data: 
    :param uri: str, 
    :param db: str, 
    :param col: str, 
    :return:
    """
    sc = spark.sparkContext
    df = prepare_dataframe(spark, sc, data)
    config = {
        'uri': uri,
        'database': db,
        'collection': col
    }
    if not df.isEmpty():
        df.write.format("mongo") \
            .options(**config) \
            .mode("append") \
            .save()

def write_to_database(spark, data, conf,
                   db: str, write_table: str, type: str = 'mariadb') -> None:
    """
    write data into database table
    :param data:
    :param uri:
    :param db:
    :param table:
    :param type:
    :return:
    """
    sc = spark.sparkContext
    df = prepare_dataframe(spark, sc, data)
    properties = get_property(conf)

    jdbc = ''
    if type == 'mariadb':
        jdbc = gen_maria_jdbc(conf, db)
    elif type == 'postgres':
        jdbc = gen_postgres_jdbc(conf, db)
    elif type == 'mssql':
        jdbc = gen_maria_jdbc(conf, db)
    else:
        raise ValueError('error when generating jdbc')

    if not df.isEmpty():
        df.write.jdbc(
            url=jdbc,
            table=write_table,
            mode="append",
            properties=properties
        )
    else:
        print('empty dataset')

def sentiment_analysis(data, tokenizer, model) -> pd.DataFrame:
    """
    sentiment analysis and retrun dataframe with score
    """
    data['sentiment'] = None
    for idx, text in enumerate(data['text']):
        tokenized_news = tokenizer(text, truncation=True, return_tensors="tf")
        logits = model.predict(tokenized_news).logits
        probabilities = tf.nn.softmax(logits)
        data.at[idx, 'sentiment'] = probabilities.numpy()
    return data

In [4]:
with open('conf.yaml', 'r') as file:
        config = yaml.safe_load(file)

mongo_uri = gen_mongo_uri(config['mongodb'])

In [5]:
try:
    # getting the spark instance
    spark = SparkSession.builder \
        .appName('Big Data Project ETL') \
        .config("spark.mongodb.input.uri", mongo_uri) \
        .config("spark.mongodb.output.uri", mongo_uri) \
        .config("spark.jars.packages", "com.microsoft.azure:spark-mssql-connector_2.12:1.2.0,"
                                       "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
        .getOrCreate()
except PySparkException as e:
    print(f"Failed to get or create Spark: {e}")
except Exception as e:
    print(f'Exception Caught: {e}')

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml

In [6]:
stock_symbols = [
        'AAPL', 'MSFT', 'NVDA', 'META', 'AMZN', 'TSLA', 'GOOGL',
        'ON', 'DBD', 'DSGX', 'GTLB', 'LOGI', 'CRSR',
        'LNG', 'SWN', 'APA', 'BTU', 'CL',
        'BMY', 'THC', 'TNDM',
        'MOS', 'AXTA', 'KOP',
        'SBLK', 'EME', 'DNOW',
    ]


In [7]:
# load historical data from database
maria_hist_data_df = spark.read.jdbc(
    url=gen_maria_jdbc(config['mariadb']),
    table='yf_historical_data',
    properties=get_property(config['mariadb'])
)

In [8]:
# load sentiment data from csv
news_sp_df = spark.read.options(delimiter=",", header=True) \
                    .csv(os.path.join('data/news_sentiment_all.csv'))

In [9]:
maria_hist_data_df.columns, news_sp_df.columns, maria_hist_data_df.count(), news_sp_df.count()

(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'symbol'],
 ['_c0', 'symbol', 'date', 'sentiment'],
 116277,
 1511)

In [10]:
maria_hist_data_df.Date

Column<'Date'>

In [11]:
maria_hist_data_df = maria_hist_data_df.withColumn('Date', to_date('Date')).withColumnRenamed('Date', 'date')
news_sp_df = news_sp_df.withColumn('date', to_date('date'))
maria_hist_data_df.show(5)

+----------+-------------------+-------------------+-------------------+-------------------+-------------------+----------+------+
|      date|               Open|               High|                Low|              Close|          Adj Close|    Volume|symbol|
+----------+-------------------+-------------------+-------------------+-------------------+-------------------+----------+------+
|2004-03-03|0.42142900824546814| 0.4319640100002289|0.42142900824546814|0.42714300751686096| 0.3616190552711487| 225131200|  AAPL|
|2004-03-04| 0.4273209869861603|0.45035699009895325|0.42696401476860046|0.44928601384162903| 0.3803653120994568| 660223200|  AAPL|
|2004-03-05| 0.4455359876155853|0.49089300632476807| 0.4446429908275604|0.47749999165534973| 0.4042512774467468|1540599200|  AAPL|
|2004-03-08| 0.4753569960594177|0.47839298844337463| 0.4607140123844147| 0.4642859995365143|0.39306432008743286| 522872000|  AAPL|
|2004-03-09| 0.4625000059604645|0.48625001311302185| 0.4598209857940674|0.483929008

In [12]:
news_sp_df.show(5)

+---+------+----------+---------+
|_c0|symbol|      date|sentiment|
+---+------+----------+---------+
|  0|  AAAU|2024-03-02|        2|
|  1|  AAPL|2024-02-27|        1|
|  2|  AAPL|2024-02-28|        1|
|  3|  AAPL|2024-02-29|        1|
|  4|  AAPL|2024-03-01|        1|
+---+------+----------+---------+
only showing top 5 rows



In [13]:
# join news_sp_df with historical data
joined_df = maria_hist_data_df.join(news_sp_df, on=[maria_hist_data_df.symbol==news_sp_df.symbol, 
                                                    maria_hist_data_df.date==news_sp_df.date], 
                                    how='left') \
                                .select(maria_hist_data_df.symbol,
                                        maria_hist_data_df.date,
                                        maria_hist_data_df.Close,
                                        maria_hist_data_df.Volume,
                                        news_sp_df.sentiment)

In [14]:
joined_df.columns, joined_df.count()

(['symbol', 'date', 'Close', 'Volume', 'sentiment'], 116277)

In [15]:
# write result data into mariadb
write_to_database(spark, joined_df, config['mariadb'],
                   db='finance_out', write_table='historical_with_sentiment', type = 'mariadb')

data is spark df and not empty


In [16]:
spark.stop()